In [1]:
from HPC_lib import *

In [ ]:
## RE-LEARNING ##############################################


## Define parameters for experiment ##########################
n_arenas, dd_precision, n_rep_per_model = [10, 10, 10]      ##
n_grid, n_lec, n_DG, n_CA3, n_CA1 = [70, 230, 150, 80, 150] ##
##############################################################


epochs_mean = []
epochs_std = []
epochs_sem = []
loss_mean = []
loss_std = []
loss_sem = []
loss_curves = []

for i in np.arange(n_arenas):
    
    print ( str(i+1) + '/' + str(n_arenas) )
    
    ## Create arenas, model and fit 
    data = create_arena(n_grid=n_grid, n_lec=n_lec)
    model = create_model(n_DG=n_DG, n_CA3=n_CA3, n_CA1=n_CA1, dim=n_grid+n_lec)
    history = fit(model, data, save_weights=True)
    
    data2 = create_arena(n_grid=n_grid, n_lec=n_lec)
    
    epochs_mean_temp = []
    epochs_std_temp = []
    epochs_sem_temp = []
    loss_mean_temp = []
    loss_std_temp = []
    loss_sem_temp = []
    loss_curves_temp = []
    
    ## Modify the arena by a proportion of dd 
    for dd in np.linspace(0, 1, dd_precision+1):

        epochs_log = []
        loss_log = []
        loss_curve_log = []
        
        ## Average across experiments 
        for _ in np.arange(n_rep_per_model):

            new_data = modify_arena(data=data, data2=data2, dd=dd, n_grid=n_grid, n_lec=n_lec)
        
            model.load_weights('model.h5')
            history = fit(model, new_data, save_weights=False)

            epochs_log.append(len(history.history['loss']))
            loss_log.append(history.history['loss'][-1])
            loss_curve_log.append([history.history['loss']])
    
        epochs_mean_temp.append(np.mean(epochs_log))
        epochs_std_temp.append(np.std(epochs_log))
        epochs_sem_temp.append(stats.sem(epochs_log))
        loss_mean_temp.append(np.mean(loss_log))
        loss_std_temp.append(np.std(loss_log))
        loss_sem_temp.append(stats.sem(loss_log))
        loss_curves_temp.append([np.mean(loss_curve_log, axis=0)])
        
    epochs_mean.append([epochs_mean_temp])
    epochs_std.append([epochs_std_temp])
    epochs_sem.append([epochs_sem_temp])
    loss_mean.append([loss_mean_temp])
    loss_std.append([loss_std_temp])
    loss_sem.append([loss_sem_temp])
    loss_curves.append([loss_curves_temp])
    
epochs_mean = np.mean(epochs_mean, axis=0)
epochs_std = np.std(epochs_std, axis=0)
epochs_sem = stats.sem(epochs_sem, axis=0)
loss_mean = np.mean(loss_mean, axis=0)
loss_std = np.std(loss_std, axis=0)
loss_sem = stats.sem(loss_sem, axis=0)
loss_curves = np.mean(loss_curves, axis=0)

1/10


In [ ]:
plt.figure(figsize=(16,8))
loss_curves = loss_curves.reshape((11,50))

begin = 0
for dd in np.arange(1,11):
    plt.axvline(x=begin, linestyle='--', c='black', linewidth=1.)
    plt.plot(np.arange(begin, begin+len(loss_curves[dd])), loss_curves[dd], linewidth=5.)
    begin += len(loss_curves[dd])

plt.title('Adaptation after context modification', fontsize=28)
#plt.xlabel('Epochs', fontsize=20)
#plt.xticks(fontsize=20)
plt.xlabel('Context modification (%)', fontsize=20)
plt.xticks(np.linspace(0,500,11), np.round(np.linspace(0.1,1,10),1), fontsize=20)
plt.tick_params(axis=u'both', which=u'both',length=0)
plt.ylabel('Loss', fontsize=20)
plt.yticks(fontsize=20)
plt.ylim(0,0.02)  # appropriate scale for comparison with initial learning would be (0, 0.045)

plt.plot(np.linspace(50,500,10), loss_curves[1:,-1], color='black', linewidth=0.5, marker='o')
plt.plot(np.linspace(0,450,10), loss_curves[1:,0], color='black', linewidth=0.5, marker='o')